### 런타임 유형 변경 > A100 및 대용량 RAM 활성화 후 1번 셀 실행

1번 셀을 실행하면 충돌을 유발하는 캐시를 비운 뒤
런타임이 강제종료 됩니다.

정상작동의 결과이니 바로
FinetuningData_fixed.jsonl파일을 업로드 하고 2번 셀을 실행해주세요

### 1. 라이브러리 준비

In [ ]:
print("=== 1. 캐시 삭제 및 라이브러리 강제 설치 시작 ===")

# 1. Hugging Face 캐시를 강제로 비웁니다 (손상된 파일 다운로드 오류 해결)
print("--> Hugging Face 캐시 삭제 중...")
!rm -rf ~/.cache/huggingface/

# 2. 호환되는 특정 버전으로 재설치 (triton, bitsandbytes *제외*)
print("--> 라이브러리 강제 설치 중...")
!pip uninstall -y trl peft transformers accelerate bitsandbytes
!pip install -q transformers==4.41.2 accelerate==0.31.0 peft==0.11.1 trl==0.9.3 datasets==2.20.0

print("=== 1. 설치 완료. 커널을 강제로 재시작합니다. ===")
print("!!! [중요] 런타임이 비정상 종료되었다는 경고가 뜨면 100% 정상입니다. !!!")
print("!!! 자동으로 다시 연결될 때까지 기다렸다가, 2단계로 'FinetuningData.jsonl' 파일을 [다시] 업로드하고, 2번 셀을 실행하세요. !!!")

# 3. 커널 강제 종료 (라이브러리 로딩용)
import os
os.kill(os.getpid(), 9)

=== 1. 캐시 삭제 및 라이브러리 강제 설치 시작 ===
--> Hugging Face 캐시 삭제 중...
--> 라이브러리 강제 설치 중...
Found existing installation: trl 0.9.3
Uninstalling trl-0.9.3:
  Successfully uninstalled trl-0.9.3
Found existing installation: peft 0.11.1
Uninstalling peft-0.11.1:
  Successfully uninstalled peft-0.11.1
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0


### 2. 모델 불러오기 + 파인튜닝 시작

In [ ]:
#################################################################
# 2. 전체 파인튜닝 스크립트 (A100 + Qwen2 / 16비트 + OOM 수정)
#################################################################

print("=== 2단계: A100용 (Qwen2) 16비트(bfloat16) 파인튜닝 스크립트 시작 ===")
print("라이브러리 로드 중...")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import json

# --- 모델 및 토크나저 로드 (16비트 bfloat16) ---
print("=== 모델 및 토크나저 로드 시작 ===")
model_id = "spow12/Ko-Qwen2-7B-Instruct"
model_dtype = torch.bfloat16

# force_download=True: 캐시 문제 방지를 위해 강제 다운로드
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=None, force_download=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

#device_map="cuda:0": 모델을 즉시 GPU 0번으로 로드하여 CPU 메모리 부족 방지
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda:0", # A100에 강제 할당
    torch_dtype=model_dtype,
    cache_dir=None,
    force_download=True
)
print("=== 모델 및 토크나저 로드 완료 ===")


# --- 3. LoRA 설정 (16비트용) ---
# 2. LoRA (Low-Rank Adaptation) 설정
# 설명: 전체 파라미터를 학습하는 대신, 일부 파라미터만 학습하여 메모리를 절약하는 기법
print("=== LoRA 설정 시작 ===")
peft_config = LoraConfig(
    r=16, # LoRA Rank: 학습할 파라미터의 수 결정 (클수록 성능↑, 메모리↑)
    lora_alpha=32, # Scaling Factor: 학습 가중치 보정값 (보통 r의 2배 사용)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05, # 과적합 방지를 위한 드롭아웃
    bias="none", # 바이어스 파라미터는 학습하지 않음
    task_type="CAUSAL_LM" # 인과적 언어 모델링(다음 단어 예측) 태스크 지정
)
# 모델에 LoRA 어댑터 장착
model = get_peft_model(model, peft_config)
# 학습 가능한 파라미터 수 출력 (전체의 약 0.1~1% 수준인지 확인)
model.print_trainable_parameters()
print("=== LoRA 설정 완료 ===")


# --- 4. 데이터셋 준비  ---
print("=== 데이터셋 로드 및 포맷팅 시작 ===")
data_files = "FinetuningData_fixed.jsonl"
dataset = load_dataset("json", data_files=data_files, split="train")

"""
데이터셋의 'input'(공고+자소서)과 'output'(JSON 분석)을
LLM이 이해할 수 있는 대화형 프롬프트(Chat Template)로 변환하는 함수
"""
def formatting_prompts_func(example):
    output_dict = dict(example)
    user_input = output_dict.pop('input') # 사용자 입력 (질문)
    # 모델이 출력해야 할 정답 (JSON 문자열)
    assistant_output = json.dumps(output_dict, ensure_ascii=False)

    # 시스템 프롬프트 모델의 역할(페르소나)과 출력 형식을 강제하는 핵심 지시사항
    system_prompt = """당신은 IT 직군 전문 채용 어드바이저입니다. 채용 공고와 자기소개서를 비교 분석하여, 반드시 다음 JSON 스키마에 맞춰서만 응답해야 합니다.

{
  "score": "종합 점수(0-100 정수)",
  "strengths": [
    "자소서가 공고에 부합하는 강점 (문자열 배열)"
  ],
  "weaknesses": [
    "자소서가 공고에 비해 부족한 약점 (문자열 배열)"
  ],
  "missing_keywords": [
    "공고에는 있으나 자소서에 빠진 키워드 (문자열 배열)"
  ],
  "overall_advice": "종합적인 조언 (문자열)"
}
"""
    # Chat Template 구조 생성 (System -> User -> Assistant)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": assistant_output}
    ]
    # 토크나이저를 사용해 모델 고유의 포맷(예: <|im_start|>)으로 변환
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return { "text": text }

# 전체 데이터셋에 포맷팅 적용
dataset = dataset.map(formatting_prompts_func)
print("=== 데이터셋 로드 및 포맷팅 완료 ===")


# --- 5. 파인튜닝 트레이너 설정 (OOM 오류 수정) ---
print("=== 파인튜닝 트레이너 설정 시작 ===")

sft_config = SFTConfig(
    # SFTTrainer 고유 인수
    dataset_text_field="text", # 데이터셋에서 학습할 텍스트 컬럼명
    max_seq_length=4096, # 입력 시퀀스 최대 길이 (길면 메모리 많이 차지함)
    packing=True, # 여러 짧은 샘플을 하나로 묶어 학습 속도 향상
    dataset_num_proc=1,

    # TrainingArguments 인수 및 저장경
    output_dir="./results",

    # [OOM 오류 수정]
    # [메모리 최적화 설정 - 중요]
    # 배치 사이즈를 1로 줄여 GPU 메모리 부담을 최소화
    per_device_train_batch_size=1,
    # 대신 그래디언트를 4번 축적하여 실질적인 배치 사이즈 4 효과를 냄
    gradient_accumulation_steps=4,

    # [학습 스케줄]
    warmup_steps=5,        # 초기 학습률을 서서히 올리는 웜업 스텝
    num_train_epochs=3,    # 전체 데이터를 3번 반복 학습
    learning_rate=2e-4,    # LoRA 학습에 적합한 학습률

    # [하드웨어 가속 설정]
    bf16=True,             # A100 GPU 전용: bfloat16 사용 (메모리 절약 + 성능 향상)

    # [기타 설정]
    logging_steps=1,       # 매 스텝마다 로그 출력
    optim="adamw_torch",   # Pytorch 기본 옵티마이저 사용
    save_strategy="epoch", # 에폭마다 모델 저장
    report_to="none",      # WandB 등 외부 로깅 끄기
)

# 트레이너 객체 생성 (Supervised Fine-Tuning Trainer)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args=sft_config
)
print("=== 파인튜닝 트레이너 설정 완료 ===")


# --- 6. 학습 시작 ---
print("*********************************")
print("********* 파인튜닝 시작! *********")
print("*********************************")

trainer.train()

print("*********************************")
print("********* 파인튜닝 완료! *********")
print("*********************************")


# --- 7. 모델 저장 ---
save_path = "./my_finetuned_model_v2"
trainer.save_model(save_path)

print(f"파인튜닝된 모델 어댑터가 {save_path}에 저장되었습니다!")
print("왼쪽 파일 탭에서 'my_finetuned_model_v2' 폴더를 확인하세요.")

=== 2단계: A100용 (Qwen2) 16비트(bfloat16) 파인튜닝 스크립트 시작 ===
라이브러리 로드 중...
=== 모델 및 토크나저 로드 시작 ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Step,Training Loss
1,1.325600
2,1.485100
3,1.372500
4,1.306600
5,1.335300
6,1.387500
7,1.364200
8,1.197000
9,1.460300
10,1.374700


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


*********************************
********* 파인튜닝 완료! *********
*********************************


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


파인튜닝된 모델 어댑터가 ./my_finetuned_model_v2에 저장되었습니다!
왼쪽 파일 탭에서 'my_finetuned_model_v2' 폴더를 확인하세요.


### 3. 새로운 데이터로 성능 확인

학습이 완료된 LoRA 어댑터를 원본(Base) 모델에 결합하여,
새로운 채용 공고와 자소서에 대해 AI가 실제로 JSON 피드백을
생성하는지 검증하는 단계입니다.

In [ ]:
#################################################################
# 8. 파인튜닝된 모델로 "추론" 테스트하기
#################################################################
from peft import PeftModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

print("=== 1. 원본 모델(Ko-Qwen2) 로드 중... ===")

# --- 16비트(bfloat16)로 원본 모델 다시 로드 ---
model_id = "spow12/Ko-Qwen2-7B-Instruct"
# A100 GPU 성능 최적화를 위해 bfloat16 사용 (학습 때와 동일하게 설정)
model_dtype = torch.bfloat16

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# 원본 모델을 GPU(cuda:0)에 16비트 모드로 로드
# device_map="cuda:0": 모델을 GPU 0번에 직접 할당하여 CPU 메모리 사용 최소화
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda:0", # A100에 강제 할당
    torch_dtype=model_dtype,
)

print("=== 2. 파인튜닝 '어댑터' 로드 중... ===")

# --- 2. 7단계에서 저장한 'my_finetuned_model' 어댑터 로드 ---
#원본 모델 위에 우리가 학습시킨 추가 레이어(어댑터)를 얹습니다.
save_path = "./my_finetuned_model_v2"
# Base 모델 + LoRA 어댑터 결합
peft_model = PeftModel.from_pretrained(base_model, save_path)
# 추론 모드로 설정 (Dropout 비활성화 등)
peft_model.eval()

print("=== 3. 테스트 시작! (새로운 공고/자소서 입력) ===")

# --- 3. 학습에 사용하지 않은 "새로운" 데이터로 질문 ---
new_job_posting = """
### 채용 공고:
헤렌∙서울 성동구∙경력 2-4년

[브랜다즈] 백엔드 개발자

헤렌은 안정적인 비즈니스 모델을 통해 외부 투자 없이도 매년 50% 이상의 성장을 달성하고 있습니다.
브랜다즈팀은 헤렌만의 독보적 기술력으로 20만 고객이 이용하는 SNS 자동화 솔루션인 인스타겟, 인스타터 그리고 데이터 기반 인플루언서 마케팅 플랫폼 파인앳플을 운영하고 있습니다.

주요업무
• 브랜다즈의 서비스인 인스타겟, 인스타터의 신규 상품 관련 아키텍처를 설계 및 구현합니다.
• 인플루언서 플랫폼인 파인앳플의 구매 리뷰와 숏폼 관련 아키텍처를 설계 및 구현합니다.
• 인스타겟, 파인앳플의 CX 파트로부터 인입되는 각종 개발 관련 이슈를 주도적으로 해결합니다.
• 기획, 마케팅 파트와 긴밀하게 협업하며 서비스의 신규 기능을 빠르게 개발합니다.
자격요건
• 2년 이상의 JAVA, SpringBoot를 이용한 개발 및 운영 경험이 있으신 분
• 코드 리뷰 및 테스트 코드 작성 경험이 있으신 분
• RDBMS 설계 경험 및 데이터 모델링 경험이 있으신 분
• 기존 비즈니스를 이해하고, 보다 나은 방향으로 제안하는 데 적극적이신 분
• 원활한 커뮤니케이션으로 개발 조직 뿐 아니라 타 부서와도 원활한 협업을 하시는 분

[주요 기술스택]
Java, Kotlin, SpringBoot, Kafka, JPA/Hibernate, Postgresql, IntelliJ, DataGrip,
GitHub, Slack, Notion, AWS(ECS, EB) 등 다양한 서비스, Jenkins 등을 이용한 CI/CD 자동화
우대사항
• 1년 이상의 Kotlin을 이용한 개발 경험이 있으신 분
• 기획 참여부터 설계, 출시, 운영까지 직접 경험해 보신 분
• AI를 활용하여 업무 능력을 향상시킨 경험이 있으신 분
• 주어진 업무를 수행하는 것을 넘어 스스로 개선할 업무를 찾고, 적극적인 실행력으로 문제를 개선한 경험이 있으신 분
• 업무 프로세스를 개선해서 팀의 생산성을 높인 경험이 있으신 분
"""

new_resume = """
### 자기소개서:
저는 2년 6개월간 Java와 SpringBoot를 사용하여 커머스 백엔드 API를 개발 및 운영한 경험이 있습니다.

주로 기획팀과 협업하여 신규 이벤트와 상품 API를 개발하는 역할을 맡았습니다. JPA와 Postgresql을 사용하여 RDBMS를 설계하고, 요청받은 비즈니스 로직을 구현했습니다. 사내 코드 리뷰 문화에 따라 코드 리뷰에 참여했으며, JUnit을 사용한 단위 테스트 코드 작성 경험이 있습니다.

CX 파트에서 전달받은 이슈를 해결하고, 기획/마케팅 파트와 원활하게 커뮤니케이션하며 맡은 기능 개발을 기한 내에 완료하는 것을 중요하게 생각합니다.

비록 Kotlin이나 Kafka를 실무에 적용해본 경험은 없지만, Java/SpringBoot 개발 경험을 바탕으로 브랜다즈팀의 서비스 개발에 기여하고 싶습니다. 새로운 기술 스택도 빠르게 학습할 자신이 있습니다.
"""

# --- 4. 프롬프트 템플릿에 맞춰 질문 구성 ---

system_prompt = """당신은 IT 직군 전문 채용 어드바이저입니다. 채용 공고와 자기소개서를 비교 분석하여, 반드시 다음 JSON 스키마에 맞춰서만 응답해야 합니다.

{
  "score": "종합 점수(0-100 정수)",
  "strengths": [
    "자소서가 공고에 부합하는 강점 (문자열 배열)"
  ],
  "weaknesses": [
    "자소서가 공고에 비해 부족한 약점 (문자열 배열)"
  ],
  "missing_keywords": [
    "공고에는 있으나 자소서에 빠진 키워드 (문자열 배열)"
  ],
  "overall_advice": "종합적인 조언 (문자열)"
}
"""
# 대화형 메시지 구조 생성
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"{new_job_posting}\n{new_resume}"},
]
# [중요] 토크나이저를 사용해 모델이 이해하는 형식으로 변환
# add_generation_prompt=True: 모델이 답변을 시작할 준비를 하도록 특수 토큰 추가 (<|im_start|>assistant\n)
prompt_template = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True # <-- AI가 답변을 생성하도록 함
)

# --- 5. 모델에 질문하고 답변(JSON) 생성 ---
# 입력을 텐서로 변환하여 GPU로 이동
inputs = tokenizer(prompt_template, return_tensors="pt").to("cuda")

# 추론 실행 (No Grad: 그래디언트 계산 안함 -> 메모리 절약)
with torch.no_grad():
    output_ids = peft_model.generate(
        **inputs,
        max_new_tokens=1024,    # 생성할 최대 토큰 수 (JSON 길이에 맞춰 넉넉하게)
        do_sample=True,         # 확률적 샘플링 사용 (다양한 표현 가능)
        temperature=0.1,        # [중요] 0에 가까울수록 논리적이고 일관된(결정적인) 답변 생성 (JSON 깨짐 방지)
        top_p=0.9,              # 상위 90% 확률 내의 토큰만 선택
        pad_token_id=tokenizer.eos_token_id,
    )

# 입력 프롬프트 부분은 제외하고, 모델이 새로 생성한 부분만 디코딩
generated_ids = output_ids[0, inputs["input_ids"].shape[1]:]
output_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

# JSON 문자열을 파싱 후, indent=4를 사용하여 예쁘게 출력합니다.
try:
    # 1. 모델이 출력한 텍스트를 JSON 객체로 파싱
    parsed_json = json.loads(output_text.strip())

    # 2. JSON 객체를 4칸 들여쓰기(indent=4)로 문자열 변환 (보기 좋게)
    pretty_json = json.dumps(parsed_json, indent=4, ensure_ascii=False)

    print("="*20, " 파인튜닝된 모델의 응답 ", "="*20)
    print(pretty_json)

except json.JSONDecodeError:
    # 모델이 JSON 형식을 깨뜨렸을 경우 원본 텍스트를 출력 (디버깅 용)
    print("="*20, " JSON 파싱 오류! 원본 텍스트 출력 ", "="*20)
    print(output_text.strip())


=== 1. 원본 모델(Ko-Qwen2) 로드 중... ===


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

=== 2. 파인튜닝 '어댑터' 로드 중... ===
=== 3. 테스트 시작! (새로운 공고/자소서 입력) ===
====================  파인튜닝된 모델의 응답  ====================
{
    "score": 79,
    "strengths": [
        "2년 6개월간 Java와 SpringBoot를 사용한 개발 및 운영 경험이 있어 기본 자격요건을 충족한다.",
        "JPA와 Postgresql을 사용한 RDBMS 설계 경험이 있으며, 데이터 모델링 경험이 언급된다.",
        "코드 리뷰와 단위 테스트 코드 작성 경험이 있어 일부 자격요건을 충족한다.",
        "기획/마케팅 파트와의 협업 및 이슈 해결 경험이 있어 주요업무와 일치한다."
    ],
    "weaknesses": [
        "Kotlin, Kafka, Postgresql, IntelliJ, DataGrip, GitHub, Slack, Notion, AWS(ECS, EB), Jenkins 등 주요 기술스택 중 대부분을 실제로 사용해 본 경험은 부족하다.",
        "AI 활용 경험이나 업무 프로세스 개선 경험이 언급되지 않아 우대사항에 부합하지 않는다.",
        "1년 이상의 Kotlin 사용 경험이 없어 우대사항을 충족하지 못한다.",
        "기획 참여부터 설계, 출시, 운영까지 직접 경험해 본 경험이 언급되지 않아 우대사항에 부합하지 않는다."
    ],
    "missing_keywords": [
        "Kotlin",
        "Kafka",
        "Postgresql",
        "IntelliJ IDEA",
        "DataGrip",
        "GitHub",
        "Slack",
        "Notion",
        "AWS ECS",
        "AWS EB",
        "Jenkins",
     

4. 저장한 모델 zip파일로 생성하기

In [ ]:
# 'my_finetuned_model_v2' 폴더를 'my_finetuned_model.zip_v2' 파일로 압축합니다.
!zip -r /content/my_finetuned_model_v2.zip /content/my_finetuned_model_v2

  adding: content/my_finetuned_model_v2/ (stored 0%)
  adding: content/my_finetuned_model_v2/training_args.bin (deflated 53%)
  adding: content/my_finetuned_model_v2/tokenizer_config.json (deflated 65%)
  adding: content/my_finetuned_model_v2/adapter_config.json (deflated 53%)
  adding: content/my_finetuned_model_v2/README.md (deflated 66%)
  adding: content/my_finetuned_model_v2/adapter_model.safetensors (deflated 22%)
  adding: content/my_finetuned_model_v2/added_tokens.json (deflated 36%)
  adding: content/my_finetuned_model_v2/special_tokens_map.json (deflated 46%)
  adding: content/my_finetuned_model_v2/tokenizer.json (deflated 72%)
  adding: content/my_finetuned_model_v2/merges.txt (deflated 57%)
  adding: content/my_finetuned_model_v2/vocab.json (deflated 61%)
